# Extract data from subreddits 

### Imports

In [7]:
import praw 
import matplotlib
from dotenv import load_dotenv
import os


### Loading creds from .env

In [8]:
load_dotenv()

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")

### Connecting to Reddit API

In [ ]:
reddit = praw.Reddit(
    client_id = client_id,
    client_secret = client_secret,
    user_agent = user_agent
)

# # Testing connection
subreddit = reddit.subreddit("dataengineering")
print(f"Subreddit Title: {subreddit.title}")
print(f"Subreddit Description: {subreddit.description}")
print(f"Subreddit Subscribers: {subreddit.subscribers}")

### Ingest Data from top 10 posts for this week

In [ ]:
posts = subreddit.top(time_filter = "week", limit=10)

data = []

for post in posts:
    data.append({
        "title": post.title,
        "score": post.score,
        "url": post.url,
        "num_comments": post.num_comments,
        "created_utc": post.created_utc
    })

# Print the data
print(data)

[{'title': 'You can become a millionaire working in Data', 'score': 2658, 'url': 'https://i.redd.it/tk3ipjstr0we1.png', 'num_comments': 81, 'created_utc': 1745167815.0}, {'title': 'WTF that guy just wrote a database in 2 lines of bash', 'score': 732, 'url': 'https://i.redd.it/nqcgrkcm0twe1.png', 'num_comments': 103, 'created_utc': 1745509742.0}, {'title': 'Apache Airflow 3.0 is here – and it’s a big one!', 'score': 452, 'url': 'https://www.reddit.com/r/dataengineering/comments/1k5b6kr/apache_airflow_30_is_here_and_its_a_big_one/', 'num_comments': 56, 'created_utc': 1745340613.0}, {'title': '𝐃𝐨𝐨𝐫𝐃𝐚𝐬𝐡 𝐃𝐚𝐭𝐚 𝐓𝐞𝐜𝐡 𝐒𝐭𝐚𝐜𝐤', 'score': 253, 'url': 'https://i.redd.it/lj3dfdf3j7xe1.jpeg', 'num_comments': 24, 'created_utc': 1745685746.0}, {'title': 'Apache Airflow® 3 is Generally Available!', 'score': 124, 'url': 'https://www.reddit.com/r/dataengineering/comments/1k5bc5m/apache_airflow_3_is_generally_available/', 'num_comments': 8, 'created_utc': 1745340989.0}, {'title': 'Is the title “Data Enginee